# Validare una rete neurale
Il processo di validazione di una rete neurale è importante per evitare che un modello possa andare in contro ad overfitting sul set di test. Questo è un problema più raro rispetto all'overfitting sul set di addestramento, ma che può verficarsi quando testiamo diversi modelli alla ricerca degli iperparametri migliori.
<br>
Una soluzione per affrontare questo problema consiste nel creare un terzo set apposito per la validazione, quindi avremo i seguenti set:
 * **Set di addestramento**: contiene gli esempi utilizzati per addestrare la rete.
 * **Set di validazione**: contiene gli esempi per validare ottimizzare gli iperparametri evitando l'overfitting.
 * **Set di test**: contiene gli esempi per testare il modello finale per verificarne la capacità predittiva.
 
Vediamo come utilizzare un set di validazione utilizzando Keras e la nostra rete addestrata con l'IMDB Movie Review dataset.
<br><br>
Importiamo i moduli necessari.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.datasets import imdb 
from keras.regularizers import l2
from keras.layers import Dense, Dropout

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Carichiamo e prepariamo il dataset, inizialmente suddividendolo solo in set di addestramento e test.

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

def onehot_encoding(data, size):
    onehot = np.zeros((len(data), size))
    for i, d in enumerate(data):
        onehot[i,d] = 1.
    return onehot

X_train = onehot_encoding(X_train, 5000)
X_test = onehot_encoding(X_test, 5000)

X_train.shape

(25000, 5000)

Definiamo l'architettura della nostra rete.

In [4]:
def create_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(5000,), kernel_regularizer=l2(0.1)))
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(8,activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

Per utilizzare un set di validazione con Keras abbiamo due opzioni.

## Opzione 1: Definire manualmente un set di validazione

Creiamo il modello e compiliamolo.

In [5]:
model = create_model()
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

Creiamo un set di validazione, estraendo gli ultimi 2500 esempi dal set di addestramento.

In [7]:
X_train2 = X_train[:-2500]
y_train2 = y_train[:-2500]
X_val = X_train[-2500:]
y_val = y_train[-2500:]
print("Esempi nel set di addestramento: %d " % X_train2.shape[0])
print("Esempi nel set di validazione: %d " % X_val.shape[0])
print("Esempi nel set di test: %d " % X_test.shape[0])

Esempi nel set di addestramento: 22500 
Esempi nel set di validazione: 2500 
Esempi nel set di test: 25000 


Adesso possiamo avviare l'addestramento passando il set di addestramento così creato all'interno del parametro <span style="font-family: Monaco">validation_data</span> del metodo <span style="font-family: Monaco">fit</span>.

In [8]:
model.fit(X_train2, y_train2, epochs=100, batch_size=512, validation_data=(X_val, y_val))

Train on 22500 samples, validate on 2500 samples
Epoch 1/100
22500/22500 [==============================] - 4s 200us/step - loss: 13.4928 - acc: 0.5155 - val_loss: 1.3383 - val_acc: 0.6944
Epoch 2/100
22500/22500 [==============================] - 4s 164us/step - loss: 0.9331 - acc: 0.6074 - val_loss: 0.7812 - val_acc: 0.8148
Epoch 3/100
22500/22500 [==============================] - 4s 165us/step - loss: 0.7469 - acc: 0.7164 - val_loss: 0.6809 - val_acc: 0.8512
Epoch 4/100
22500/22500 [==============================] - 4s 162us/step - loss: 0.7185 - acc: 0.8007 - val_loss: 0.6716 - val_acc: 0.8384
Epoch 5/100
22500/22500 [==============================] - 4s 167us/step - loss: 0.7027 - acc: 0.8295 - val_loss: 0.6338 - val_acc: 0.8640
Epoch 6/100
22500/22500 [==============================] - 4s 172us/step - loss: 0.6800 - acc: 0.8359 - val_loss: 0.6181 - val_acc: 0.8604
Epoch 7/100
22500/22500 [==============================] - 4s 176us/step - loss: 0.6660 - acc: 0.8398 - val_loss: 0.

22500/22500 [==============================] - 4s 175us/step - loss: 0.4805 - acc: 0.8777 - val_loss: 0.4855 - val_acc: 0.8624
Epoch 60/100
22500/22500 [==============================] - 4s 180us/step - loss: 0.4798 - acc: 0.8752 - val_loss: 0.4699 - val_acc: 0.8820
Epoch 61/100
22500/22500 [==============================] - 4s 172us/step - loss: 0.4816 - acc: 0.8772 - val_loss: 0.4815 - val_acc: 0.8840
Epoch 62/100
22500/22500 [==============================] - 4s 179us/step - loss: 0.4724 - acc: 0.8806 - val_loss: 0.4802 - val_acc: 0.8812
Epoch 63/100
22500/22500 [==============================] - 4s 181us/step - loss: 0.4857 - acc: 0.8760 - val_loss: 0.4776 - val_acc: 0.8712
Epoch 64/100
22500/22500 [==============================] - 4s 180us/step - loss: 0.4763 - acc: 0.8789 - val_loss: 0.4757 - val_acc: 0.8760
Epoch 65/100
22500/22500 [==============================] - 4s 170us/step - loss: 0.4776 - acc: 0.8764 - val_loss: 0.4792 - val_acc: 0.8704
Epoch 66/100
22500/22500 [=======

Come puoi vedere, al termine di ogni epoca il modello viene valutato anche sul set di validazione, calcolando le metriche da noi indicate, in questo caso binary crossentropy e accuracy.
<br><br>
Infine, per valutare la capacità predittiva effettiva del modello, dobbiamo testarlo sul set di test.

In [9]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 3s 134us/step


[0.5102350357437134, 0.87224]

## Opzione 2: Estrarre automaticamente un set di validazione
La seconda opzione consiste nel lasciare che Keras estragga automaticamente un set di validazione dal set di addestramento, specificandone la dimensione. Creiamo nuovamente il modello.

In [13]:
model = create_model()
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

Adesso passiamo al metodo <span style="font-family: Monaco">fit</span>. il parametro <span style="font-family: Monaco">validation_split</span>, indicando all'interno la frazione del set di addestramento da utilizzare come set di validazione, utilizziamo il 10% specificando 0.1, quindi dato questo conterrà sempre 2500 esempi.

In [14]:
model.fit(X_train, y_train, epochs=100, batch_size=512, validation_split=0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/100
22500/22500 [==============================] - 5s 200us/step - loss: 13.5702 - acc: 0.5478 - val_loss: 1.3525 - val_acc: 0.7284
Epoch 2/100
22500/22500 [==============================] - 4s 179us/step - loss: 0.9302 - acc: 0.7036 - val_loss: 0.7166 - val_acc: 0.8496
Epoch 3/100
22500/22500 [==============================] - 3s 146us/step - loss: 0.7409 - acc: 0.7820 - val_loss: 0.6388 - val_acc: 0.8584
Epoch 4/100
22500/22500 [==============================] - 4s 184us/step - loss: 0.7081 - acc: 0.8136 - val_loss: 0.6198 - val_acc: 0.8656
Epoch 5/100
22500/22500 [==============================] - 3s 153us/step - loss: 0.7035 - acc: 0.8170 - val_loss: 0.5876 - val_acc: 0.8604
Epoch 6/100
22500/22500 [==============================] - 4s 163us/step - loss: 0.6485 - acc: 0.8355 - val_loss: 0.5777 - val_acc: 0.8716
Epoch 7/100
22500/22500 [==============================] - 4s 166us/step - loss: 0.6466 - acc: 0.8408 - val_loss: 0.

22500/22500 [==============================] - 4s 168us/step - loss: 0.4953 - acc: 0.8774 - val_loss: 0.4683 - val_acc: 0.8744
Epoch 60/100
22500/22500 [==============================] - 4s 170us/step - loss: 0.4837 - acc: 0.8804 - val_loss: 0.4637 - val_acc: 0.8824
Epoch 61/100
22500/22500 [==============================] - 7s 303us/step - loss: 0.4891 - acc: 0.8778 - val_loss: 0.4722 - val_acc: 0.8752
Epoch 62/100
22500/22500 [==============================] - 9s 386us/step - loss: 0.4919 - acc: 0.8788 - val_loss: 0.4910 - val_acc: 0.8708
Epoch 63/100
22500/22500 [==============================] - 4s 194us/step - loss: 0.4869 - acc: 0.8816 - val_loss: 0.4635 - val_acc: 0.8804
Epoch 64/100
22500/22500 [==============================] - 4s 200us/step - loss: 0.4871 - acc: 0.8796 - val_loss: 0.4706 - val_acc: 0.8824
Epoch 65/100
22500/22500 [==============================] - 9s 403us/step - loss: 0.4878 - acc: 0.8819 - val_loss: 0.4725 - val_acc: 0.8820
Epoch 66/100
22500/22500 [=======

E testiamo di nuovo sul set di test

In [15]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 8s 329us/step


[0.47609062995910645, 0.86992]